# 0. Переход на GPU

In [ ]:
# Среда выполнения → Сменить среду выполнения → Графический процессор T4

# GPU ускоряет обработку моделей и данных, делая выполнение кода в разы быстрее по сравнению с обычным процессором

# 1. Установка и загрузка библиотек

In [ ]:
# Монтируем Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Установка необходимых библиотек
!pip install --upgrade --force-reinstall numpy
!pip install -U scikit-learn bertopic pyldavis gensim

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 76.8 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.3.0 which is incompatible.
cupy-cuda12x 13.3.0 requires numpy<2.3,>=1.22, but you have numpy 2.3.0 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.3.0 which is incompatible.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 78.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.6/150.6 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 84.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 73.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 91.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 91.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 80.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664

# 2. Перезапуск среды выполнения

In [ ]:
# Среда выполнения → Перезапустить сеанс

# 3. Импорты, пути, чтение и функции для BERTopic и LDA

In [ ]:
import os
import pandas as pd

from sentence_transformers import SentenceTransformer
from bertopic import BERTopic
from umap import UMAP
from hdbscan import HDBSCAN

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from gensim.corpora.dictionary import Dictionary
import numpy as np

import joblib
import csv

from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
import pyLDAvis

paths = {
    "vk": "/content/drive/MyDrive/TextScope/processed_vk_data.csv",
    "gnews": "/content/drive/MyDrive/TextScope/processed_gnews_data.csv",
    "cyberleninka": "/content/drive/MyDrive/TextScope/processed_cyberleninka_data_keywords.csv"
}

plots_dir = "/content/drive/MyDrive/TextScope/plots"
os.makedirs(plots_dir, exist_ok=True)

# 4. Функция для интерактивного BERTopic (с вводом числа тем вручную)

In [ ]:
def bertopic_pipeline(df, text_column, out_path, source):
    print(f"Запускаем BERTopic для {source}")

    docs = df[text_column].astype(str).tolist()
    device = "cuda" if __import__('torch').cuda.is_available() else "cpu"
    embedder = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2", device=device)

    model = BERTopic(
        embedding_model=embedder,
        nr_topics="auto",
        umap_model=UMAP(n_neighbors=5, n_components=10, min_dist=0.3, metric='cosine'),
        hdbscan_model=HDBSCAN(min_cluster_size=30, min_samples=5, metric='euclidean', cluster_selection_method='leaf'),
        calculate_probabilities=False,
        verbose=False
    )
    model.fit(docs)

    print("Визуализация тем (интерактивная)...")
    fig = model.visualize_topics()
    display(HTML(fig.to_html(include_plotlyjs='cdn')))

    # Сохраняем первичную визуализацию
    html_path_initial = os.path.join(plots_dir, f"{source}_bertopic_initial.html")
    fig.write_html(html_path_initial)
    print(f"Первичная визуализация BERTopic сохранена в {html_path_initial}")

    n_topics_widget = widgets.IntText(value=10, description='Число тем:', min=2)
    button = widgets.Button(description="Применить редукцию")
    output = widgets.Output()
    progress = widgets.Label(value="Ожидание выбора пользователем...")

    def on_button_clicked(b):
        with output:
            clear_output(wait=True)
            n_topics = n_topics_widget.value
            print(f"Выбрано {n_topics} тем.")
            progress.value = "Редукция тем..."

            reduced_model = model.reduce_topics(docs, nr_topics=n_topics)
            progress.value = "Редукция завершена. Создание визуализации..."

            fig_reduced = reduced_model.visualize_topics()
            display(HTML(fig_reduced.to_html(include_plotlyjs='cdn')))

            # Сохраняем HTML в plots_dir
            plot_path = os.path.join(plots_dir, f"{source}_reduced_topics_{n_topics}.html")
            with open(plot_path, "w", encoding="utf-8") as f:
                f.write(fig_reduced.to_html(include_plotlyjs='cdn'))

            progress.value = "Сохранение CSV и модели..."

            topics = reduced_model.topics_
            df["topic"] = topics
            df.to_csv(out_path, index=False, quoting=1)

            model_path = out_path.replace(".csv", f"_{source}_model_{n_topics}")
            reduced_model.save(model_path)

            progress.value = "Файлы сохранены."
            print(f"Результаты сохранены:\nCSV: {out_path}\nМодель: {model_path}\nГрафик: {plot_path}")

    button.on_click(on_button_clicked)
    display(n_topics_widget, button, progress, output)

# 5. Функция для LDA с ручным вводом числа тем

In [ ]:
# Для Cyberleninka используем LDA по ключевым словам.
# Это связано с тем, что тексты представляют собой длинные научные статьи,
# и BERTopic может работать с ними нестабильно.
# Использование предварительно выделенных keywords делает модель LDA более точной и быстрой.

In [ ]:
def lda_pipeline(df, keyword_column, out_path, source):
    print(f"Запускаем LDA для {source}")

    df = df[df[keyword_column].notna()]
    df = df[df[keyword_column].str.strip().astype(bool)]

    texts = []
    for kw_str in df[keyword_column]:
        try:
            kw_list = eval(kw_str)
            if isinstance(kw_list, list) and kw_list:
                texts.append(kw_list)
        except:
            continue
    df = df.iloc[:len(texts)].copy()

    id2word = Dictionary(texts)
    corpus_gensim = [id2word.doc2bow(text) for text in texts]

    texts_joined = [" ".join(t) for t in texts]
    vectorizer = CountVectorizer(min_df=1)
    dtm = vectorizer.fit_transform(texts_joined)

    # Предварительное построение модели со 100 темами
    default_n_topics = 100
    lda = LatentDirichletAllocation(
        n_components=default_n_topics,
        learning_method='online',
        batch_size=512,
        random_state=42
    )
    lda.fit(dtm)

    topic_term_dists = lda.components_ / lda.components_.sum(axis=1)[:, np.newaxis]
    doc_topic_dists = lda.transform(dtm)
    doc_lengths = dtm.sum(axis=1).A1
    vocab = vectorizer.get_feature_names_out()
    term_frequency = dtm.sum(axis=0).A1

    vis_data = pyLDAvis.prepare(
        topic_term_dists,
        doc_topic_dists,
        doc_lengths,
        vocab,
        term_frequency
    )
    html_str = pyLDAvis.prepared_data_to_html(vis_data)
    display(HTML(html_str))

    # Сохраняем первичную визуализацию
    html_path_initial = os.path.join(plots_dir, f"{source}_lda_initial_{default_n_topics}.html")
    with open(html_path_initial, 'w') as f:
        f.write(html_str)
    print(f"Первичная визуализация LDA сохранена в {html_path_initial}")

    # Виджеты для задания нового числа тем
    n_topics_widget = widgets.IntText(value=10, description='Число тем:', min=2)
    button = widgets.Button(description="Запустить LDA c новым числом тем")
    output = widgets.Output()

    def on_button_clicked(b):
        with output:
            clear_output(wait=True)
            n_topics = n_topics_widget.value
            print(f"Выбрано {n_topics} тем.")

            lda = LatentDirichletAllocation(
                n_components=n_topics,
                learning_method='online',
                batch_size=512,
                random_state=42
            )
            lda.fit(dtm)

            topic_term_dists = lda.components_ / lda.components_.sum(axis=1)[:, np.newaxis]
            doc_topic_dists = lda.transform(dtm)
            doc_lengths = dtm.sum(axis=1).A1
            vocab = vectorizer.get_feature_names_out()
            term_frequency = dtm.sum(axis=0).A1

            vis_data_2 = pyLDAvis.prepare(
                topic_term_dists,
                doc_topic_dists,
                doc_lengths,
                vocab,
                term_frequency
            )
            html_str_2 = pyLDAvis.prepared_data_to_html(vis_data_2)
            display(HTML(html_str_2))

            # Сохраняем финальную визуализацию
            html_path = os.path.join(plots_dir, f"{source}_lda_{n_topics}_topics.html")
            with open(html_path, 'w') as f:
                f.write(html_str_2)
            print(f"Финальная визуализация LDA сохранена в {html_path}")

            # Сохраняем модель и результаты
            topic_distributions = lda.transform(dtm)
            df['topic'] = topic_distributions.argmax(axis=1)

            df.to_csv(out_path, index=False, quoting=csv.QUOTE_ALL, quotechar='"')

            model_dir = out_path.replace(".csv", f"_{source}_lda_model")
            os.makedirs(model_dir, exist_ok=True)
            joblib.dump(lda, os.path.join(model_dir, "lda_model.joblib"))
            joblib.dump(vectorizer, os.path.join(model_dir, "vectorizer.joblib"))
            id2word.save(os.path.join(model_dir, "id2word.dict"))

            print(f"LDA результаты сохранены:\nCSV: {out_path}\nМодель: {model_dir}")

    button.on_click(on_button_clicked)
    display(n_topics_widget, button, output)

# 6. Основной запуск — по каждому файлу вызываем нужный пайплайн

## Инструкция по запуску обработки тем

Для корректной работы пайплайна необходимо выполнять обработку **каждого источника отдельно**, следуя этим шагам:

1. **Запустите первую ячейку** (например, для источника `vk`) и дождитесь появления интерактивного графика.
2. Посмотрите график и в появившемся поле **введите желаемое количество тем**.
3. Дождитесь завершения обработки — появится сообщение о сохранении результатов (CSV, модель, визуализация).
4. После этого переходите к следующей ячейке (например, для источника `gnews`) и повторите шаги **1–3**.
5. Аналогично выполните обработку для всех других источников.

> **Важно:** Не используйте кнопку "Выполнить всё" (`Run all`) — обработка требует вашего ввода (`input`) после каждого источника.


In [ ]:
key = 'vk'
path = paths[key]
out_path = path.replace(".csv", "_topics.csv")

if os.path.exists(out_path):
    print(f"{out_path} уже существует — пропускаем.")
else:
    print(f"Обрабатываем файл: {path}")
    df = pd.read_csv(path)
    bertopic_pipeline(df, text_column="text", out_path=out_path, source=key)

Обрабатываем файл: /content/drive/MyDrive/TextScope/processed_vk_data.csv
Запускаем BERTopic для vk


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.89k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

/usr/local/lib/python3.11/dist-packages/ipywidgets/widgets/widget.py:477: DeprecationWarning:

Passing unrecognized arguments to super(IntText).__init__(min=2).
object.__init__() takes exactly one argument (the instance to initialize)
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.



IntText(value=10, description='Число тем:')

Button(description='Применить редукцию', style=ButtonStyle())

Label(value='Ожидание выбора пользователем...')

Output()

In [ ]:
key = 'gnews'
path = paths[key]
out_path = path.replace(".csv", "_topics.csv")

if os.path.exists(out_path):
    print(f"{out_path} уже существует — пропускаем.")
else:
    print(f"Обрабатываем файл: {path}")
    df = pd.read_csv(path)
    bertopic_pipeline(df, text_column="text", out_path=out_path, source=key)

Обрабатываем файл: /content/drive/MyDrive/TextScope/processed_gnews_data.csv
Запускаем BERTopic для gnews
Визуализация тем (интерактивная)...


Первичная визуализация BERTopic сохранена в /content/drive/MyDrive/TextScope/plots/gnews_bertopic_initial.html


/usr/local/lib/python3.11/dist-packages/ipywidgets/widgets/widget.py:477: DeprecationWarning:

Passing unrecognized arguments to super(IntText).__init__(min=2).
object.__init__() takes exactly one argument (the instance to initialize)
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.



IntText(value=10, description='Число тем:')

Button(description='Применить редукцию', style=ButtonStyle())

Label(value='Ожидание выбора пользователем...')

Output()

In [ ]:
key = 'cyberleninka'
path = paths[key]
out_path = path.replace(".csv", "_topics.csv")

if os.path.exists(out_path):
    print(f"{out_path} уже существует — пропускаем.")
else:
    print(f"Обрабатываем файл: {path}")
    df = pd.read_csv(path)
    lda_pipeline(df, keyword_column="keywords", out_path=out_path, source=key)

Обрабатываем файл: /content/drive/MyDrive/TextScope/processed_cyberleninka_data_keywords.csv
Запускаем LDA для cyberleninka


Первичная визуализация LDA сохранена в /content/drive/MyDrive/TextScope/plots/cyberleninka_lda_initial_100.html


/usr/local/lib/python3.11/dist-packages/ipywidgets/widgets/widget.py:477: DeprecationWarning:

Passing unrecognized arguments to super(IntText).__init__(min=2).
object.__init__() takes exactly one argument (the instance to initialize)
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.



IntText(value=10, description='Число тем:')

Button(description='Запустить LDA c новым числом тем', style=ButtonStyle())

Output()



---



# Просмотр топ-слов по темам

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip install bertopic

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.6/150.6 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 84.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 64.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 77.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-

In [ ]:
import glob
from bertopic import BERTopic

# Пути к CSV-файлам в Google Drive
paths = {
    "vk": "/content/drive/MyDrive/TextScope/processed_vk_data_topics.csv",
    "gnews": "/content/drive/MyDrive/TextScope/processed_gnews_data_topics.csv"
}

# Функция для поиска пути к модели
def find_bertopic_model(path_csv, source):
    pattern = path_csv.replace(".csv", f"_{source}_model_*")
    matches = glob.glob(pattern)
    if matches:
        print(f"Найден путь для {source}: {matches[0]}")
        return matches[0]
    else:
        raise FileNotFoundError(f"BERTopic model for {source} not found.")

# Функция для печати топ-слов тем
def print_top_words(model_path, source, n_words=10):
    model = BERTopic.load(model_path)
    topics_info = model.get_topic_info()
    print(f"\nИсточник: {source} — список тем и ключевых слов:\n")

    for topic_id in topics_info['Topic']:
        if topic_id == -1:
            continue  # пропуск аутлайеров
        top_words = model.get_topic(topic_id)
        words = [word for word, _ in top_words[:n_words]]
        print(f"Тема {topic_id}: {', '.join(words)}")

# Основной запуск
for source, csv_path in paths.items():
    try:
        model_path = find_bertopic_model(csv_path, source)
        print_top_words(model_path, source)
    except FileNotFoundError as e:
        print(e)

Найден путь для vk: /content/drive/MyDrive/TextScope/processed_vk_data_topics_vk_model_14

Источник: vk — список тем и ключевых слов:

Тема 0: андерсен, тихвинский, кренкель, огурец, астероид, берингов, перец, чудотворный, усопший, телескоп
Тема 1: вебинар, кейс, многоступенчатый, счастливчик, онлайнэтап, инноватор, побороться, рандомайзер, подарочный, анонимизировать
Тема 2: циркон, гиперзвуковой, оглядываться, авианосец, искандер, гитара, отнекиваться, фичь, просыпать, лётчиковистребитель
Тема 3: зефирка, энергостратегия, гидрометслужба, навесный, тумба, подвесной, настенный, антресоль, биом, микропластика
Тема 4: nan, мирно, анапа, методично, новичок, разлив, впечатлять, международныйотбор, краеугольный, мирнинский
Тема 5: казино, бонус, промокод, преференциальный, исковый, губернаторский, самозанятой, демченко, букмекерский, инвестпроект
Тема 6: магнитная, метеозависимый, геошторм, раздражительность, недомогание, первомай, багрянец, кандинский, дебоширить, разъярённый
Тема 7: галак

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
import joblib

def print_cyberleninka_top_words(model_dir, n_words=10):
    lda = joblib.load(f"{model_dir}/lda_model.joblib")
    vectorizer = joblib.load(f"{model_dir}/vectorizer.joblib")

    vocab = vectorizer.get_feature_names_out()

    print("cyberleninka: список тем и ключевых слов:\n")
    for topic_idx, topic in enumerate(lda.components_):
        top_features_ind = topic.argsort()[:-n_words - 1:-1]
        top_words = [vocab[i] for i in top_features_ind]
        print(f"Тема {topic_idx}: {', '.join(top_words)}")

print_cyberleninka_top_words("/content/drive/MyDrive/TextScope/processed_cyberleninka_data_keywords_topics_cyberleninka_lda_model")

cyberleninka: список тем и ключевых слов:

Тема 0: осадки, суша, северовосток, боковой, буровая, целиков, жила, марка, целик, замер
Тема 1: северовосток, юрцев, арбитражный, бердяев, хронотоп, худяков, убор, абрамов, лебёдка, соцветие
Тема 2: авроральный, магнитная, самуил, магнитосфера, климатологический, межпланетный, северовосток, аэрономия, прихожанин, осадки
Тема 3: осадки, северовосток, выводок, минтай, берингов, пологий, геном, моховой, петроглиф, обтекание
Тема 4: столбовой, зерновые, раскоп, специализированный, северовосток, математика, осадки, судопроизводство, суша, венчик
Тема 5: берингов, суша, специализированный, норд, разлив, северовосток, избиратель, производная, енисейхатангский, среднедушевой
Тема 6: северовосток, топонимия, миссионерский, индейский, розвиток, саами, зодчество, ргиа, берингов, хантыйский
Тема 7: послеоперационный, котлован, неврологический, пациентка, железа, геострофический, биостанция, эсер, эпителий, уединённый
